<a href="https://colab.research.google.com/github/Rohan-Bha/Special-Topcis-MIS-IntroToAI2026/blob/main/Lab_3_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install openai -q
from openai import OpenAI
import os, re

client = OpenAI(api_key="API_Key")
print('Setup complete.')

Setup complete.


In [7]:
REACT_SYSTEM_PROMPT = """You are an expert Python developer using the ReACT framework.
You MUST follow this exact cycle for every task:

[THOUGHT]: Reason about the problem. Identify inputs, outputs, edge cases, and approach.
[PLAN]: List the steps you will take, numbered. Be specific.
[CODE]: Write the complete, runnable Python code. Wrap it in ```python ... ``` fences.
[EXPECTED_OUTPUT]: Describe what the output should look like when it runs correctly.
[EDGE_CASES]: List at least 2 edge cases your code handles.

Code constraints:
- Use ONLY pandas and the Python standard library.
- The function must be named analyze_csv(filepath: str) -> str
- It must RETURN a formatted multi-line string report, NOT print directly.
- Handle FileNotFoundError and empty DataFrame gracefully with informative error messages.
- Handle columns with all-NaN values without crashing.
- Include inline comments explaining non-obvious logic.
- The report must include: column name, mean, median, std dev, and missing value count for each numeric column.
- Target Python 3.8+ compatibility."""

print('System prompt defined.')

System prompt defined.


In [8]:
REACT_USER_PROMPT = """Task: Write a Python function that analyzes a CSV file and returns a formatted statistics report.

The function signature must be:
    analyze_csv(filepath: str) -> str

The report string must contain:
1. Total rows and columns in the CSV
2. For each numeric column: mean, median, std dev, min, max, and count of missing values
3. A summary line listing any columns that have >10% missing values
4. A footer with the total number of missing cells across the whole dataframe

Follow the [THOUGHT] → [PLAN] → [CODE] → [EXPECTED_OUTPUT] → [EDGE_CASES] format exactly."""

print('User prompt defined.')

User prompt defined.


In [9]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': REACT_SYSTEM_PROMPT},
        {'role': 'user', 'content': REACT_USER_PROMPT}
    ],
    temperature=0.2,
    max_tokens=1500
)

react_output = response.choices[0].message.content
print('=== ReACT OUTPUT (Thought + Plan + Code) ===')
print(react_output)

=== ReACT OUTPUT (Thought + Plan + Code) ===
[THOUGHT]: The task requires analyzing a CSV file and generating a report on its numeric columns. The inputs will be the file path of the CSV, and the outputs will be a formatted string report. Edge cases include handling missing files, empty DataFrames, and columns with all NaN values. The approach will involve reading the CSV using pandas, calculating the required statistics, and formatting the output string.

[PLAN]:
1. Import the necessary libraries (pandas).
2. Define the function `analyze_csv(filepath: str) -> str`.
3. Use a try-except block to handle FileNotFoundError when reading the CSV file.
4. Read the CSV file into a DataFrame.
5. Check if the DataFrame is empty and return an appropriate message if it is.
6. Calculate the total number of rows and columns.
7. Initialize a list to store statistics for each numeric column.
8. Iterate over each numeric column to calculate mean, median, std dev, min, max, and count of missing values.


In [10]:
def extract_code(text):
    pattern = r'```python\s*([\s\S]*?)```'
    matches = re.findall(pattern, text)
    if not matches:
        raise ValueError('No Python code block found in AI output.')
    return matches[0].strip()

generated_code = extract_code(react_output)
print('=== EXTRACTED CODE ===')
print(generated_code)

=== EXTRACTED CODE ===
import pandas as pd

def analyze_csv(filepath: str) -> str:
    try:
        # Attempt to read the CSV file into a DataFrame
        df = pd.read_csv(filepath)
    except FileNotFoundError:
        return f"Error: The file '{filepath}' was not found."
    
    # Check if the DataFrame is empty
    if df.empty:
        return "Error: The DataFrame is empty."
    
    # Get total rows and columns
    total_rows, total_columns = df.shape
    report_lines = [f"Total Rows: {total_rows}, Total Columns: {total_columns}"]
    
    # Initialize a list to track columns with >10% missing values
    missing_value_summary = []
    total_missing_cells = df.isnull().sum().sum()  # Total missing cells in the DataFrame
    
    # Iterate over numeric columns to gather statistics
    for column in df.select_dtypes(include=['number']).columns:
        # Calculate statistics
        mean = df[column].mean()
        median = df[column].median()
        std_dev = df[column].std()
    

In [11]:
test_csv_content = """name,age,salary,score,department
Alice,30,75000,88.5,Engineering
Bob,25,,92.0,Marketing
Carol,35,82000,,Engineering
David,,69000,78.3,Sales
Eve,28,91000,95.1,Engineering
Frank,42,110000,81.7,Management
Grace,31,,88.2,Marketing
Henry,29,77000,,Sales
Iris,38,85000,90.4,Engineering
Jack,,72000,76.8,Sales
"""

with open('test_data.csv', 'w') as f:
    f.write(test_csv_content)

print('Test CSV created!')
print('Preview:')
import pandas as pd
print(pd.read_csv('test_data.csv').to_string())

Test CSV created!
Preview:
    name   age    salary  score   department
0  Alice  30.0   75000.0   88.5  Engineering
1    Bob  25.0       NaN   92.0    Marketing
2  Carol  35.0   82000.0    NaN  Engineering
3  David   NaN   69000.0   78.3        Sales
4    Eve  28.0   91000.0   95.1  Engineering
5  Frank  42.0  110000.0   81.7   Management
6  Grace  31.0       NaN   88.2    Marketing
7  Henry  29.0   77000.0    NaN        Sales
8   Iris  38.0   85000.0   90.4  Engineering
9   Jack   NaN   72000.0   76.8        Sales


In [12]:
exec_namespace = {}

try:
    exec(generated_code, exec_namespace)
    analyze_fn = exec_namespace.get('analyze_csv')

    if analyze_fn is None:
        raise NameError('Function analyze_csv not found in generated code.')

    result = analyze_fn('test_data.csv')
    print('=== EXECUTION OUTPUT ===')
    print(result)
    print('\n✅ Code executed successfully!')
    execution_error = None

except Exception as e:
    print(f'❌ Error: {e}')
    execution_error = str(e)

=== EXECUTION OUTPUT ===
Total Rows: 10, Total Columns: 5
Column: age, Mean: 32.25, Median: 30.50, Std Dev: 5.65, Min: 25.0, Max: 42.0, Missing Values: 2
Column: salary, Mean: 82625.00, Median: 79500.00, Std Dev: 13168.55, Min: 69000.0, Max: 110000.0, Missing Values: 2
Column: score, Mean: 86.37, Median: 88.35, Std Dev: 6.66, Min: 76.8, Max: 95.1, Missing Values: 2
Columns with >10% missing values: age, salary, score
Total Missing Cells: 6

✅ Code executed successfully!


In [13]:
if execution_error:
    FIX_PROMPT = f"""The code you generated produced this error when executed:

ERROR: {execution_error}

CODE THAT FAILED:
```python
{generated_code}
```

[THOUGHT]: Diagnose what went wrong.
[FIX]: Explain the fix you will apply.
[CODE]: Provide the complete corrected function in ```python ... ``` fences."""

    fix_response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {'role': 'system', 'content': REACT_SYSTEM_PROMPT},
            {'role': 'user', 'content': REACT_USER_PROMPT},
            {'role': 'assistant', 'content': react_output},
            {'role': 'user', 'content': FIX_PROMPT}
        ],
        temperature=0.2,
        max_tokens=1200
    )

    fix_output = fix_response.choices[0].message.content
    print('=== FIX ITERATION OUTPUT ===')
    print(fix_output)

    fixed_code = extract_code(fix_output)
    exec_namespace2 = {}
    exec(fixed_code, exec_namespace2)
    result = exec_namespace2['analyze_csv']('test_data.csv')
    print('\n=== FIXED CODE OUTPUT ===')
    print(result)
    print('\n✅ Fixed and working!')

else:
    print('✅ No fix needed — code worked on the first try!')

✅ No fix needed — code worked on the first try!


In [14]:
print('--- Edge Case 1: File not found ---')
try:
    print(exec_namespace['analyze_csv']('nonexistent.csv'))
except Exception as e:
    print(f'Error: {e}')

print('\n--- Edge Case 2: Empty CSV (headers only, no data) ---')
with open('empty.csv', 'w') as f:
    f.write('name,age,salary\n')

try:
    print(exec_namespace['analyze_csv']('empty.csv'))
except Exception as e:
    print(f'Error: {e}')

--- Edge Case 1: File not found ---
Error: The file 'nonexistent.csv' was not found.

--- Edge Case 2: Empty CSV (headers only, no data) ---
Error: The DataFrame is empty.
